In [1]:
import shutil
import os
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!cp -u /content/drive/MyDrive/small_dataset_sorted_by_cat.zip ./
!unzip -n small_dataset_sorted_by_cat.zip

Archive:  small_dataset_sorted_by_cat.zip
   creating: small_dataset_sorted_by_cat/
   creating: small_dataset_sorted_by_cat/4468/
   creating: small_dataset_sorted_by_cat/4284/
   creating: small_dataset_sorted_by_cat/2840/
   creating: small_dataset_sorted_by_cat/4438/
   creating: small_dataset_sorted_by_cat/2206/
  inflating: small_dataset_sorted_by_cat/4468/19-M.jpg  
  inflating: small_dataset_sorted_by_cat/4468/8-1.jpg  
  inflating: small_dataset_sorted_by_cat/4468/8-M.jpg  
  inflating: small_dataset_sorted_by_cat/4468/30-M.jpg  
  inflating: small_dataset_sorted_by_cat/4468/19-1.jpg  
  inflating: small_dataset_sorted_by_cat/4284/32-3.jpg  
  inflating: small_dataset_sorted_by_cat/4284/32-2.jpg  
  inflating: small_dataset_sorted_by_cat/4284/32-1.jpg  
  inflating: small_dataset_sorted_by_cat/4284/32-4.jpg  
  inflating: small_dataset_sorted_by_cat/4284/32-M.jpg  
  inflating: small_dataset_sorted_by_cat/2840/11-1.jpg  
  inflating: small_dataset_sorted_by_cat/2840/26-M.jpg  

In [6]:
!pip install torch
!pip install torchvision
!pip install pytorch-lightning

     |████████████████████████████████| 916 kB 8.2 MB/s 
     |████████████████████████████████| 829 kB 12.2 MB/s 
     |████████████████████████████████| 118 kB 39.9 MB/s 
     |████████████████████████████████| 272 kB 38.6 MB/s 
     |████████████████████████████████| 636 kB 40.1 MB/s 
     |████████████████████████████████| 1.3 MB 44.3 MB/s 
     |████████████████████████████████| 142 kB 72.0 MB/s 
     |████████████████████████████████| 294 kB 59.7 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=18391eb0f2e7ffb5c8e1ed992d7f2360fc84b8235044ea4e79bc0d703b5e3f33
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-0

In [7]:
from torchvision import datasets, transforms, models

from pytorch_lightning import LightningDataModule, LightningModule, Trainer
from pytorch_lightning.metrics.functional import accuracy
from pytorch_lightning.callbacks import ModelCheckpoint

import torch
from torch.nn import functional
from torch.utils.data import DataLoader, random_split

In [8]:
class ClassifyByCatDM(LightningDataModule):
    def __init__(self, setupdir, train_frac=0.9, seed=0, batch_size=64):
        
        super().__init__()

        self.batch_size = batch_size
        self.setupdir = setupdir
        self.train_frac = train_frac
        self.seed = seed
        self.batch_size = batch_size
        
        self.transform = transforms.Compose([
              transforms.Resize(size=256),
              transforms.CenterCrop(size=224),
              transforms.ToTensor(),
              transforms.Normalize([0.485, 0.456, 0.406],
                                   [0.229, 0.224, 0.225])
        ])

    def setup(self):
        
        torch.manual_seed(self.seed)
        
        dataset = datasets.ImageFolder(self.setupdir)
        self.num_classes = len(dataset.classes)
        
        set_len = len(dataset)
        train_len = int(set_len * self.train_frac)
        val_len = int(set_len * (1 - self.train_frac) / 2)
        test_len = set_len - train_len - val_len
        
        self.train, self.val, self.test = random_split(dataset, 
                                                      [train_len,
                                                       val_len,
                                                       test_len])
        self.train.dataset.transform = self.transform
        
        self.val.dataset.transform = self.transform
        
        self.test.dataset.transform = self.transform
        
    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test, batch_size=self.batch_size)

In [9]:
class ClassifyModel(LightningModule):
    def __init__(self, input_shape, num_classes,
                 learning_rate = 1e-4, batch_size=64):
        
        super().__init__()

        self.batch_size = batch_size
        
        self.save_hyperparameters()
        self.learning_rate = learning_rate
        self.dim = input_shape
        self.num_classes = num_classes
        
        self.feature_extractor = models.resnet34(pretrained=True)
        self.feature_extractor.eval()
        
        n_sizes = self._get_conv_output(input_shape)
        self.classifier = torch.nn.Linear(n_sizes, num_classes)
        
        self.predictions = []

    def _get_conv_output(self, shape):
        
        batch_size = 1
        inp = torch.autograd.Variable(torch.rand(batch_size, *shape))
        
        features = self._forward_features(inp)
        n_size = features.data.view(batch_size, -1).size(1)
        return n_size
    
    def _forward_features(self, x):
        
        x = self.feature_extractor(x)
        return x
    
    def forward(self, x):
        
        x = self._forward_features(x)
        x = x.view(x.size(0), -1)
        x = functional.log_softmax(self.classifier(x), dim=1)
        
        return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = functional.nll_loss(logits, y)

        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)
        self.log('train_loss', loss, on_step=True, on_epoch=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, logger=True)        

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = functional.nll_loss(logits, y)

        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = functional.nll_loss(logits, y)

        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)
        
        for i in range(len(y)):
            self.predictions.append(preds[i])

        self.log('test_loss', loss, prog_bar=True)
        self.log('test_acc', acc, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [13]:
batch_size = 2
dm = ClassifyByCatDM(setupdir='small_dataset_sorted_by_cat', train_frac=0.5,
                  seed=0, batch_size=batch_size)
dm.setup()

In [14]:
num_classes = dm.num_classes
model = ClassifyModel((3,224,224), num_classes,
                      batch_size=batch_size, learning_rate=2e-4)

In [17]:
checkpoint = ModelCheckpoint(dirpath='drive/MyDrive/checkpoints', monitor='val_loss', save_top_k=1)

trainer = Trainer(max_epochs=4,
                  progress_bar_refresh_rate=1,
                  gpus=[0],
                  callbacks = [checkpoint])

trainer.fit(model, dm)

trainer.test()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:446: UserWarning: Checkpoint directory drive/MyDrive/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:424: LightningDeprecationWarning: DataModule.prepare_data has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.prepare_data.
  f"DataModule.{name} has already been called, so it will not be called again. "
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:424: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  f"DataModule.{name} has already been c

Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:323: UserWarning: The number of training samples (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:424: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  f"DataModule.{name} has already been called, so it will not be called again. "
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.7142857313156128, 'test_loss': 3.169888973236084}
--------------------------------------------------------------------------------


[{'test_acc': 0.7142857313156128, 'test_loss': 3.169888973236084}]